In [25]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [2]:
# read in our data
happy = pd.read_csv("/Users/Philip/Schools/TAMU/STAT_636/homework/happiness.csv")
# keep only the columns we care about
happy = happy[happy.columns[range(4,10+1)]]
happy.head()


,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [10]:

X_train = happy[happy.columns[range(0,6)]]
# generate intercept column - this will have the same length as the other column
one_col = pd.DataFrame(np.ones(X_train.shape[0]))
one_col.rename(columns={0:'int'}, inplace=True  )

# merge these
X_train = pd.concat([one_col, X_train], axis=1)

# response
Y_train = pd.DataFrame(happy[happy.columns[6]])

print(X_train.head())
print("\n\n")
print(Y_train.head())

   int  Economy (GDP per Capita)   Family  Health (Life Expectancy)  Freedom  \
0  1.0                   1.44178  1.16374                   0.79504  0.57941   
1  1.0                   1.52733  1.14524                   0.86303  0.58557   
2  1.0                   1.42666  1.18326                   0.86733  0.56624   
3  1.0                   1.57744  1.12690                   0.79579  0.59609   
4  1.0                   1.40598  1.13464                   0.81091  0.57104   

   Trust (Government Corruption)  Generosity  
0                        0.44453     0.36171  
1                        0.41203     0.28083  
2                        0.14975     0.47678  
3                        0.35776     0.37895  
4                        0.41004     0.25492  



   Dystopia Residual
0            2.73939
1            2.69463
2            2.83137
3            2.66465
4            2.82596


In [11]:
# create a loop to execute LOOCV

def loocv_lm(X_train, Y_train):
    """Conducts Leave-One-Out Cross Validation
    
    Estimates the Mean-Squared Error for a linear model of a fixed specification.  
    
    Args:
        X_train: pandas dataframe with predictor variables.
        Y_train: pandas dataframe with corresponding response variable.
        
    Returns:
        mse_list: List containing all MSE values from all fitted models.  
        mse_tot: Average MSE from all models.
    
    """
    a = range(X_train.shape[0])
    mse_list = []

    for i in range(X_train.shape[0]):
        # design matrix
        loo_dat = X_train.iloc[a[:i] + a[(i+1):], :]
        valid_dat = X_train.iloc[a[i], :]
        valid_dat = valid_dat.values.reshape(1, -1) # manipulation required for sklearn predictions

        # response vector
        loo_resp = Y_train.iloc[a[:i] + a[(i+1):], :]
        valid_resp = Y_train.iloc[a[i], :]

        loo_regr = linear_model.LinearRegression(fit_intercept=False) # we have already handled this
        loo_regr.fit(loo_dat, loo_resp)

        # make a prediction
        loo_pred = loo_regr.predict(valid_dat)

        # calculate the mean-squared error
        mse = mean_squared_error(valid_resp, loo_pred)
        mse_list.append(mse)

    # get the final mse for all of the models    
    mse_tot = sum(mse_list) / len(mse_list)
    
    return mse_tot, mse_list

In [15]:
mse_tot, mse_list = loocv_lm(X_train=X_train, Y_train=Y_train)
print(mse_tot)

0.306910236066


In [42]:
def loo_bootstrap(X_train, Y_train, B=1000):
    """Conducts bootstrap sampling for MSE statistic.
    
    Args:
        X_train: pandas dataframe with predictor variables.
        Y_train: pandas dataframe with corresponding response variable.
        B: Number of bootstrap replicates.  Defaults to 1000.
        
    Returns:
        mse_list: List with B MSE values
        
    """
    
    # holding shell
    a = range(X_train.shape[0])
    mse_list = []
    
    for i in range(0, B):
        np.random.seed(1738+i)
        samp_idx = np.random.choice(a, size=X_train.shape[0], replace=True)
        hold_mat = X_train.iloc[samp_idx]
        hold_resp = Y_train.iloc[samp_idx]
        
        bs_regr = linear_model.LinearRegression(fit_intercept=False)
        bs_regr.fit(hold_mat, hold_resp)
        
        bs_pred = bs_regr.predict(hold_mat)
        
        mse = mean_squared_error(hold_resp, bs_pred)
        mse_list.append(mse)
    
    return mse_list    

In [46]:
bs_mse = loo_bootstrap(X_train=X_train, Y_train=Y_train, B=1000)
np.std(bs_mse)

0.033067136039092296

In [ ]:
boot = loo_bootstrap(x=mse_list)
# compare
print(sum(boot) / len(boot))
print(mse_tot)

In [48]:
# let's make a class

class HomeworkFive:
    """Performs both LOOCV and bootstrapping for MSE statistic. 
    
    Methods:
        loocv_lm: Performs leave-one-out CV on two pandas df inputs.
        loo_bootstrap: Conducts bootstrap sampling for MSE statistic.
        
    """
    
    def loocv_lm(self, X_train, Y_train):
        """Conducts Leave-One-Out Cross Validation

        Estimates the Mean-Squared Error for a linear model of a fixed specification.  

        Args:
            X_train: pandas dataframe with predictor variables.
            Y_train: pandas dataframe with corresponding response variable.

        Returns:
            mse_list: List containing all MSE values from all fitted models.  
            mse_tot: Average MSE from all models.

        """
        a = range(X_train.shape[0])
        mse_list = []

        for i in range(X_train.shape[0]):
            # design matrix
            loo_dat = X_train.iloc[a[:i] + a[(i+1):], :]
            valid_dat = X_train.iloc[a[i], :]
            valid_dat = valid_dat.values.reshape(1, -1) # manipulation required for sklearn predictions

            # response vector
            loo_resp = Y_train.iloc[a[:i] + a[(i+1):], :]
            valid_resp = Y_train.iloc[a[i], :]

            loo_regr = linear_model.LinearRegression(fit_intercept=False) # we have already handled this
            loo_regr.fit(loo_dat, loo_resp)

            # make a prediction
            loo_pred = loo_regr.predict(valid_dat)

            # calculate the mean-squared error
            mse = mean_squared_error(valid_resp, loo_pred)
            mse_list.append(mse)

        # get the final mse for all of the models    
        mse_tot = sum(mse_list) / len(mse_list)

        return mse_tot, mse_list
        
    
    def loo_bootstrap(self, X_train, Y_train, B=1000):
        """Conducts bootstrap sampling for MSE statistic.

        Args:
            X_train: pandas dataframe with predictor variables.
            Y_train: pandas dataframe with corresponding response variable.
            B: Number of bootstrap replicates.  Defaults to 1000.

        Returns:
            mse_list: List with B MSE values

        """

        # holding shell
        a = range(X_train.shape[0])
        mse_list = []

        for i in range(0, B):
            np.random.seed(1738+i)
            samp_idx = np.random.choice(a, size=X_train.shape[0], replace=True)
            hold_mat = X_train.iloc[samp_idx]
            hold_resp = Y_train.iloc[samp_idx]

            bs_regr = linear_model.LinearRegression(fit_intercept=False)
            bs_regr.fit(hold_mat, hold_resp)

            bs_pred = bs_regr.predict(hold_mat)

            mse = mean_squared_error(hold_resp, bs_pred)
            mse_list.append(mse)

        return mse_list    
    
    

In [55]:
phil = HomeworkFive()
mse_tot, mse_list = phil.loocv_lm(X_train=X_train, Y_train=Y_train)
print("LOOCV MSE: ", mse_tot)
print("\n") 
mse_list = phil.loo_bootstrap(X_train=X_train, Y_train=Y_train)
print("LOOCV MSE SD: " ,np.std(mse_list))

('LOOCV MSE: ', 0.30691023606597351)


('LOOCV MSE SD: ', 0.033067136039092296)


In [ ]:
# regularized regression

# first, determine the proper alpha
opt_alpha = linear_model.LassoCV(fit_intercept=False)
opt_alpha.fit(X_train, Y_train)
print(opt_alpha.alpha_)


In [ ]:
# now, make a function to do this for us:
def loocv_lasso(X_train, Y_train):

    a = range(X_train.shape[0])
    mse_list = []

    for i in range(X_train.shape[0]):
        # design matrix
        loo_dat = X_train.iloc[a[:i] + a[(i+1):], :]
        valid_dat = X_train.iloc[a[i], :]
        valid_dat = valid_dat.values.reshape(1, -1) # manipulation required for sklearn predictions

        # response vector
        loo_resp = Y_train.iloc[a[:i] + a[(i+1):], :]
        valid_resp = Y_train.iloc[a[i], :]

        loo_regr = linear_model.Lasso(alpha=opt_alpha.alpha_ , fit_intercept=False) # we have already handled this
        loo_regr.fit(loo_dat, loo_resp)

        # make a prediction
        loo_pred = loo_regr.predict(valid_dat)

        # calculate the mean-squared error
        mse = mean_squared_error(valid_resp, loo_pred)
        mse_list.append(mse)
    
    mse_tot = sum(mse_list) / len(mse_list)
    return mse_tot, mse_list

In [ ]:
lasso_mse_tot, lasso_mse_list = loocv_lasso(X_train, Y_train)

In [ ]:
print(lasso_mse_tot)

In [ ]:
class Bag:
    def __init__(self):
        self.data = []
    
    def add(self, x):
        self.data.append(x)
        
    def addtwice(self, x):
        self.add(x)
        self.add(x)

phil = Bag()

phil.addtwice(3)
print phil.data

In [ ]:
d = Dog('Fido')
d.add_trick('roll over')
print(d.tricks)
d.add_trick('play_dead')
print(d.tricks)

In [ ]:
while phil.counter < 10:
    phil.counter = phil.counter * 2
    print(phil.counter)
print(phil.counter)
del(phil.counter)


    